### Import Keras module that will be used later for our computation and save a few useful constants

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras import optimizers


Using TensorFlow backend.


In [3]:
img_width, img_height = 256, 256
batch_size=16
epochs=50
train_data_dir='/media/dheeraj/9A26F0CB26F0AA01/WORK/jupyter_files/dogs-vs-cats/train'
test_data_dir='/media/dheeraj/9A26F0CB26F0AA01/WORK/jupyter_files/dogs-vs-cats/test'
#OUT Categories
out_categories=1

#number of train, test samples
num_train_samples=25000
#num_test_samples=



### Load the pretrained model on the imagenet VGG16 network and omit the last layer because we will add custom classification network on the top of the prebuilt VGG16 and replace the last classification layer

In [4]:
#load the VGG16 model pretrained on imagenet
base_model=applications.VGG16(weights="imagenet", 
                              include_top=False, input_shape=(img_width, img_height, 3))
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

### Freez the last 15 lowest layers for the pretrained VGG16 network.

In [6]:
for layer in base_model.layers[:15]:
    layer.trainable=False

### Add a custom set of top layers for classification

In [ ]:
top_model=Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
